In [ ]:
#i "nuget: /home/peter/work/FsChat/nuget.local"
#r "nuget: FsChat.Interactive, 0.0.1"

#r "nuget: dotenv.net, 3.2.0"
open dotenv.net
let options = DotEnvOptions(envFilePaths=[| ".env" |])
DotEnv.Load(options)
open FsChat

In [ ]:
let style = """<style>
    .chat-bubble {
        display: flex;
        flex-direction: column;
        width: 100%;
        max-width: 400px;
        margin: 0 auto;
    }
    .chat-bubble>p {
        max-width: 70%;
        padding: 10px 15px;
        margin: 10px;
        border-radius: 15px;
        position: relative;
        word-wrap: break-word;
    }
    .chat-bubble.left>p {
        align-self: flex-start;
        background-color: #e1bee7;
        border: 2px solid #ce93d8;
        color: #6a1b9a;
    }
    .chat-bubble.right>p {
        align-self: flex-end;
        background-color: #c8e6c9;
        border: 2px solid #a5d6a7;
        color: #2e7d32;
    }
</style>"""

In [ ]:
let agent1 = Chat(
    model=Gpt4o_mini,
    renderer=NotebookRenderer(props="class='chat-bubble left'", css=style),
    context=[
        System """
            You're playing the 20 questions game.
            Your role is to ask 'Is it a ...' questions with Yes-or-No answers
            in order to narrow down your options and guess the word that I'm thinking of.
            ----
            Hint: You're guessing an animal.
        """
    ])

let agent2 = Chat(
    model=Gpt4o_mini,
    renderer=NotebookRenderer(props="class='chat-bubble right'", css=style),
    context=[
        System """
            You're playing the 20 questions game.
            Your role is the one who thinks of a word and responds to my questions
            with simple "yes" or "no" answers (no additional text).
            If necessary, you can also say "Both yes and no" or "I can’t answer that".
            Once I guess the correct word, respond with “CORRECT.”
            ----
            The word I'll be trying to guess is: parrot.
        """
    ])

let rec play timesLeft (text:string) =
    if timesLeft=0 then
        printfn "Game over"
    else
        let quess = agent1.send(text).text
        let assess = agent2.send(quess).text
        if assess.Contains "CORRECT." then
            ()
        else
            play (timesLeft-1) assess

play 20 "I have a word! Which word is it? Ask the first question."

Is it a mammal?

No.

Is it a bird?

Yes.

Is it a flightless bird?

No.

Is it a bird that is commonly kept as a pet?

Yes.

Is it a parrot?

CORRECT.